# Notebook  de Relaciones de entidades

1. Preprocesar la data

In [1]:
import spacy
from collections import Counter
import pandas as pd
import re
import unidecode
from nltk.tokenize import sent_tokenize,word_tokenize
from tqdm import tqdm
nlp =spacy.load('es_core_news_md')

In [2]:
#Preprocesamiento de las oraciones
def trans(text):
    temp=text.split(' ')
    for k,i in enumerate(temp):
        i=re.sub(r'[“”´!?¡¿]*','',i)
        i=re.sub(r'[0-9]*','',i)
        #i=unidecode.unidecode(i)
        #print(i)
        if i.isupper():
            if i=='DE' or i=='LA'or i=='DEL' or i=='EL':
                i=i.lower()
            firstl=i[0]
            otros=''
            for j in range(1,len(i)):
                otros=otros+i[j].lower()
            palabra=firstl+otros
            #if palabra.lower() in stop_words:
            #    palabra=''
            temp[k]=palabra
        else:
            #if i.lower() in stop_words:
            #   i=''
            temp[k]=i
    prueba=' '.join(temp)
    return prueba 
    #print(prueba)     

In [3]:
def hallarr(listaent):
    # se separa las personas y locaciones
    persona=[]
    ubicacion=[]
    listafinal=[] #se almacena la locacion y las personas que pertenecen a la misma oracion
    for i in listaent:
        if i.label_=='PER':
            persona.append(i)
        if i.label_=='LOC':
            ubicacion.append(i)
    # se extrae las locaciones y personas que pertenecen a una misma oracion
    for j in ubicacion:
        listemporal=[] #lista temporal 
        listemporal.append(j)
        for i in persona:
            if str(j.doc)==str(i.doc):
                listemporal.append(i)
        if len(listemporal)>1:
            listafinal.append(listemporal)
    # Obtener lista unica de personas y locaciones :v
    persona2=[]
    ubicacion2=[]
    for i in persona:
        persona2.append(i.text)
    for j in ubicacion:
        ubicacion2.append(j.text)
    persona2=list(set(persona2))
    ubicacion2=list(set(ubicacion2))       
    # Unir las locaciones repetidas
    listafinalv2=[]
    listaperf=[]
    for  i in ubicacion2:
        #listatr=[]
        listatr=i
        listperl=[]
        for j in listafinal:
            if j!=[]:
                if i==str(j[0]):
                    j.pop(0)
                    listperl.extend(j)
        listafinalv2.append(listatr)
        listaperf.append(listperl)
    # Unir locaciones con nombre similar
    # Se hizo estos for para no hacer muy complejo el codigo
    maxo=[]
    for  i in listafinalv2:
        listae=[]
        for j, k in enumerate(listafinalv2):
            tr=re.findall(i,k)
            if tr!=[]:
                dpro=[j]
                if dpro in maxo:
                    ind=maxo.index(dpro)
                    maxo.pop(ind)
                listae.extend(dpro)
        maxo.append(listae)
    pruelife=[]
    for i in maxo:
        temps=[]
        perso=[]
        for j in i:
            temps=listafinalv2[i[0]]
            ste=[]
            for l in listaperf[j]:
                ste.append(str(l)) # se usa str porque no es un string
                ste=list(set(ste))
            perso.extend(ste)
            perso=list(set(perso))
        pruelife.append((temps,perso))
    # se retorna la lista con las locaciones y sus personas y las lista unicas
    return pruelife, ubicacion2, persona2

In [4]:
def listadep6(lisg,doc):
    oracion6=re.sub(r'[,;:.]+','',str(doc))
    porlo=word_tokenize(oracion6)
    for i in lisg:
        dela=word_tokenize(str(i))
        listapro=[]
        #print(dela)
        #print(lisg[0].sent)
        for l in dela:
            for p,k in enumerate(porlo):
                if l==k:
                    listapro.append(p)
        listapro=list(set(listapro))
        listapro2=[]
        for l in listapro:
            count=0
            for k in listapro:
                if abs(l-k)<len(dela):
                    count=count+1
            if count==len(dela):
                listapro2.append(l)
        if listapro2!=[]:
            lp=listapro2[len(listapro2)-1]
            texto=''
            texto=str(i)
            texto=texto.strip()
            porlo[lp]=texto
            for l in range(len(listapro2)-1):
                listapro2[l]=lp+(l+1-len(listapro2))
            listapro2.pop(len(listapro2)-1)
            #print(porlo)
            ##print(listapro)
            listafin=[]
            for l in range(len(porlo)):
                if l in listapro2:
                    pass
                else:
                    listafin.append(porlo[l])
            porlo=listafin
        #print(porlo)
    #print(porlo)
    return porlo

In [5]:
def listadep7(doc):
    listaoword=[]
    for token in doc:
        if token.ent_type_=='':
            a=str(token).split()
            listaoword.extend(a)
        else:
            listaoword.append(str(token))
    return listaoword
#listadep7(per[0].doc)

In [7]:
def proceso(listaprueba):
    listemp=[]
    listatuo=[]
    listadepubi={'obj','nsubj','nmod'}
    listadepper={'appos','nsubj','obj'}
    for k in listaprueba:
        k=k.replace('.','')
        doc=nlp(k)
        ubi=[]
        per=[]
        listauni=[]
        b=list(doc.noun_chunks)
        for noun_phrase in b:
            noun_phrase.merge(noun_phrase.root.dep_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_)
        for token in doc:
            if token.ent_type_=='LOC':
                ubi.append(token)
                listemp.append([token,listaprueba[0]])
                listauni.append(token)
            elif token.ent_type_=='PER':
                per.append(token)
                listemp.append([token,listaprueba[0]])
                listauni.append(token)
        if len(ubi)>0 and len(per)>0:
            porlo=listadep7(per[0].doc)
            for p1 in ubi:
                for p2 in per:
                    if p1.dep_ in listadepubi and p2.dep_ in listadepper:
                        if abs(porlo.index(str(p1)) - porlo.index(str(p2))) <21:
                            print(p1.dep_,p2.dep_,p1,p2)
                            listarelat=[str(p1),str(p2),k,listaprueba[0]]
                            listatuo.append(listarelat)
    return listatuo, listemp

In [10]:
#df=pd.read_excel('Prueba de relatos.xlsx')
listatecnica=[]
for capitulo in tqdm(range(0,20)):
    o=df['Texto'][capitulo]
    o=re.sub('…','...',o)
    o=o.replace('...','.')
    listaprueba=sent_tokenize(o)
    for j,i in enumerate(listaprueba):
        if j==0:
            i=i.split('\n')
            i=i[0]
        i=i.split('\n')
        i=' '.join(i)
        i=trans(i)
        listaprueba[j]=i.strip()
    listatuo2,listatemp1=proceso(listaprueba)
    listatecnica.extend(listatuo2)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

nmod obj Lima Taboga
nmod obj Las Pulgas Nicolás
nmod obj Segura Montero Pa
nmod appos Lima José Santos
nmod nsubj Lima zaqueo
nmod appos la Cruz Gómez
nmod appos la Cruz Manuela Villarán de Placencia
nmod appos Hospicio Caramanduca caliente


  5%|████▏                                                                              | 1/20 [01:11<22:39, 71.57s/it]

nmod appos Hospicio Rafael Ramírez
nsubj appos el General iglesista Rafael Ramírez
nmod appos la calle Rafael Ramírez
nmod appos la Amargura Rafael Ramírez
nmod appos Moquegua Jlt
nmod nsubj la Higuera Luis Freundt
nmod appos la Higuera Francisca Diez Canseco de Castilla
nmod nsubj Lombardo Andrés Avelino Aramburú
nmod appos Bravo el Alemán
nmod appos Denegrí Sarah Bernhardt


 10%|████████▎                                                                          | 2/20 [02:30<22:10, 73.91s/it]

...

 90%|████████████████████████████████████████████████████████████████████████▉        | 18/20 [27:42<03:33, 106.87s/it]

nmod nsubj Venezuela Leocadio Guzmán
nmod nsubj Ecuador Leocadio Guzmán
nmod appos Febres Coronel Suarez
nmod obj Milagro Aparicio
nmod appos Alcalde doña
nmod obj Alcalde María Araoz
nmod appos Alcalde Rufino Torrico
nmod appos Lima doña
nmod obj Lima María Araoz
nmod appos Lima Rufino Torrico
nmod nsubj la cercana Capilla Lorenzo Cubillas
nmod appos la calle Pancho Zegarra
nmod appos Lima Pancho Zegarra
nsubj appos el Comisario Fernández
nmod nsubj la Veracruz Carlitos
nmod appos Lima don
nmod appos Lima José
nmod nsubj Lima Pepito Gálvez
nmod appos Norte Francisco Crosby
nmod appos Norte Mario Arrús


 95%|████████████████████████████████████████████████████████████████████████████▉    | 19/20 [32:57<02:49, 169.16s/it]

nmod nsubj Carmen alto Mas
nmod appos Carmen alto Borda
nmod nsubj Tarma Figúrense
nmod obj esta Plaza Mayor Mariacha
nmod obj la Merced el sueco Young
nsubj nsubj el Burdeos Palo Seco
nmod nsubj Rin Palo Seco
nmod appos la Magdalena Francisco Raggio
nmod appos la Venecia José Bernales
nmod appos la Venecia Domingo del Solar
nmod appos la Venecia Lizardo Montero
nmod appos la Palma Peña
nsubj appos misia Venancia Peña
nsubj obj misia Venancia Guichard
nmod nsubj Ayacucho Pedro Castre
nmod nsubj Ayacucho Angelino Chaves


100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [34:36<00:00, 103.82s/it]


In [11]:

#nsubj appos
#obj nsubj 
#nsubj nsubj
#nsubj obj
#appos nsubj
#aposs obj
#nmod obj
df3=pd.DataFrame(listatecnica)

In [12]:
df3.to_excel('relacionesconectadas3.xlsx')
#df3=pd.read_excel('relacionesconectadas.xlsx')

In [9]:
df=pd.read_excel('Relatos_viajes-Benvenutto.xlsx')


In [34]:
df['Texto'][1]

'LA PLACITA RECOLETANA se une a la plazuela de la Salud por el “Callejón Largo suigeneris callecita y media, formada en su lado izquierdo por una casa vieja y húmeda con zaguán y patio en forma de rampa, que sigue a la casona solariega de los Gómez Córdova, y por una hilera de tenduchos en los que se alberga gente pobre, mulatos en su mayoría.\nEn la esquina de la Cruz hay una pulpería inevitable en todas las esquinas de nuestra ciudad propiedad de un italiano genovés de alguno de los innumerables pueblecitos de la “costa levante” o de la “ponente”, que se expresa en una jerga pintoresca empedrada de “Dio mío”, “ma”, “saprisi” y “madonna” y que en mangas de camisa o con blusa marinera comparte le trabajo de la venta y de las “yapas” con sus dependientes: dos “pichines” coloradotes y risueños.\nComo buen garibaldino quizá uno de los mil don Bartolo ha colocado en la testera el armazón dos baratas oleografías de Víctor Manuel uniforme de general, condecoraciones rutilantes, grandes mosta

In [19]:
listemp=[]
for capitulo in tqdm(range(0,20)):
    o=df['Texto'][capitulo]
    o=re.sub('…','...',o)
    o=o.replace('...','.')
    listaprueba=sent_tokenize(o)
    for j,i in enumerate(listaprueba):
        if j==0:
            i=i.split('\n')
            i=i[0]
        i=i.split('\n')
        i=' '.join(i)
        i=trans(i)
        listaprueba[j]=i.strip()
    for k in listaprueba:
        k=k.replace('.','')
        k=k.replace(':','')
        doc=nlp(k)
        b=list(doc.noun_chunks)
        for noun_phrase in b:
            noun_phrase.merge(noun_phrase.root.dep_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_)
        for token in doc:
            if token.ent_type_!='':
                #print(token)
                listemp.append([token,listaprueba[0]])

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [20:26<00:00, 61.34s/it]


In [20]:
locaciones=[]
personas=[]
for i in listemp:
    if i[0].ent_type_=='LOC':
        locaciones.append(i)
    if i[0].ent_type_=='PER':
        personas.append(i)

In [21]:
len(personas)
len(locaciones)

1315

In [87]:
#personas "appos,nmod,nsubj,obj,flat"
#ubicaciones nmod


'appos'

In [29]:
df5=pd.DataFrame(personas)
df5.to_excel('personas.xlsx')

In [51]:
df3[df3['Loc']=='la Micheo']

,Unnamed: 0,Loc,Pers,Oracion,Plazuela
46,93,la Micheo,Chávez,Don Francisco Romo y el señor Benavides que vi...,La Plazuela de la Micheo
51,101,la Micheo,Manuel Gálvez,Siguen hasta la mitad de la calle una serie de...,La Plazuela de Belén


In [7]:
df4=pd.read_excel('C:/Users/Ryu/Documents/GitHub/Relatos/Visualization/Plazuelas.xlsx')

In [19]:
lista=[]
for i in range(len(df4)):
    lista.append('Chávez')


In [20]:
df4['Personajes']=lista

In [25]:
#df4['Personajes'][15]='Felipe Barreda'
df4['Personajes'][13]='Chávez, Manuel Gálvez'

<ipython-input-25-d9d881779c2b>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
import plotly.express as px
listazero=[10]*len(df4)
fig = px.scatter_mapbox(df4, lat="lat", lon="lon", hover_name="Plazuelas",hover_data=['Lugar actual','Personajes'],
                        color_discrete_sequence=["red"], zoom=12, height=700, size=listazero)        
fig.update_layout(mapbox_style="open-street-map")

In [65]:
listalocs=df3['Loc'].tolist()

In [67]:
listalocs=list(set(listalocs))

In [83]:
listaer=[]
for i in listalocs:
    a=df3[df3['Loc']==i]
    perso=a['Pers'].tolist()
    listad=[i,perso, len(perso)]
    listaer.append(listad)

In [84]:
dfg=pd.DataFrame(listaer,columns=['Loc','Personas','Cant'])

In [89]:
dfg.sort_values('Cant',ascending=False)

,Loc,Personas,Cant
13,Lima,"[José Santos, Don Carlos, el vecino Alty, Gail...",8
31,Boza,"[ don Carlos Paz Soldán, Pedro Paz Soldán, Man...",4
5,la Merced,"[Malmborg, el sueco Young, Malmborg]",3
9,la Condesa,"[Félix Zoppi, ño Cerezo]",2
20,la Micheo,"[Chávez, Manuel Gálvez]",2
...,...,...,...
29,San Pedro,[Montalbán],1
30,Colegio de la O,[Pedro Drinot],1
1,Panteón de Lima,[Matías Maestro],1
32,San Juan de Dios,[Juan de Arona],1


In [92]:
print(dfg['Personas'][31])

[' don Carlos Paz Soldán', 'Pedro Paz Soldán', 'Manuel González de la Rosa', 'Mariano Felipe']
